# 1. SymPy

In [24]:
import sympy as sym
from sspipe import p, px

In [2]:
x, y, z = sym.symbols('x,y,z')

In [6]:
expr = sym.cos(x) + 1
expr.subs(x, 'pi')

0

In [9]:
expr = sym.cos(x) + 1
expr.subs(x, 5)

cos(5) + 1

In [ ]:
expr = sym.cos(x) + 1
expr.subs(x, 5).evalf()

1.28366218546323

In [12]:
expr = x * y + z
expr.subs([(x, 2), (y, 3), (z, 4)])

10

In [16]:
expr = sym.sin(2*x)
sym.expand_trig(expr)

2*sin(x)*cos(x)

In [25]:
sym.log(x)*x**2 | p(sym.Integral)

Integral(x**2*log(x), x)

In [23]:
sym.gamma(x)

gamma(x)

In [21]:
sym.factor(x**2 - 4*x + 3)

(x - 3)*(x - 1)

# 1. Limits

## 1.1. Preview

In [1]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use(['seaborn', 'seaborn-whitegrid'])
%config InlineBackend.figure_format = 'retina'

### The velocity problem

### The area problem

# Derivatives

# Integrals

# References
- *openstax.org - [Calculus Volume 1](https://openstax.org/books/calculus-volume-1/pages/1-introduction)*
- *openstax.org - [Calculus Volume 2](https://openstax.org/books/calculus-volume-2/pages/1-introduction)*
- *openstax.org - [Calculus Volume 3](https://openstax.org/books/calculus-volume-3/pages/1-introduction)*
- *rodrigopacios.github.io - [Thomas' Calculus](https://rodrigopacios.github.io/mrpacios/download/Thomas_Calculus.pdf)*